In [1]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, regexp_tokenize
from nltk.stem import SnowballStemmer
from collections import Counter
import numpy as np

In [129]:
def procesar(texto):
  stop_words = set(stopwords.words('spanish'))

  # Tokenizar la frase
  #tokens = word_tokenize(texto)
  patron = r'\w+|[^\w\s]'
  tokens = regexp_tokenize(texto, patron)
  stemmer = SnowballStemmer('spanish')

  # Eliminar las palabras vacías
  tokens_limpios =  [ stemmer.stem(token.lower()) for token in tokens if token.isalnum() and token not in stop_words ]
  
  return tokens_limpios

In [127]:
def similitud_coseno(query, document):
    query_tokens = procesar(query)
    document_tokens = procesar(document)
    
    # Calcular la frecuencia de las palabras en la consulta y el documento
    query_freq = dict(nltk.FreqDist(query_tokens))
    document_freq = dict(nltk.FreqDist(document_tokens))
        
    # Obtener un conjunto único de todas las palabras en la consulta y el documento
    all_words = set(query_freq.keys()).union(set(document_freq.keys()))
    
    # Crear vectores de frecuencia para la consulta y el documento
    query_vector = np.array([query_freq.get(word, 0) for word in all_words])
    document_vector = np.array([document_freq.get(word, 0) for word in all_words])
    
    # Calcular la similitud coseno entre los vectores de frecuencia
    dot_product = np.dot(query_vector, document_vector)
    query_norm = np.linalg.norm(query_vector)
    document_norm = np.linalg.norm(document_vector)
    similarity = dot_product / (query_norm * document_norm)
    
    return similarity

In [131]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

def procesar2(texto):
    stop_words = set(stopwords.words('spanish'))

    # Tokenizar la frase
    tokens = word_tokenize(texto)
    stemmer = SnowballStemmer('spanish')

    # Eliminar las palabras vacías y aplicar stemming
    tokens_limpios =  [ stemmer.stem(token.lower()) for token in tokens if token.isalnum() and token not in stop_words ]

    texto_procesado = ' '.join(tokens_limpios)
    return texto_procesado

def similitud_coseno2(query, document):
    # Preprocesar los textos
    query_procesado = procesar2(query)
    document_procesado = procesar2(document)

    # Crea el vectorizador de características
    vectorizer = CountVectorizer()

    # Vectoriza los textos
    vector_query = vectorizer.fit_transform([query_procesado])
    vector_document = vectorizer.transform([document_procesado])

    # Calcula la similitud coseno entre los vectores de frecuencia
    similarity = cosine_similarity(vector_query, vector_document)

    return similarity[0][0]

In [135]:
query = "peliculas de accion y aventura"
doc = "pelicula de accion donde los personajes viven grandes aventuras por todo el mundo como si fuese una pelicula"

sm1 = similitud_coseno(query, doc)
sm2 = similitud_coseno2(query, doc)


if sm1 == sm2: print("Son iguales")
sm1, sm2

(0.6963106238227914, 0.9428090415820636)

In [ ]:
import math

def calculate_idfs(vocabulary, doc_features):
  # crea un diccionario vacío para almacenar los IDFs
  doc_idfs = {}
  # itera por cada término del vocabulario
  for term in vocabulary:
    doc_count = 0
    # itera por cada documento en el conjunto de documentos
    for doc_id in doc_features.keys():
      # obtiene los términos del documento actual
      terms = doc_features.get(doc_id)
      # si el término actual aparece en el documento actual, aumenta el contador
      if term in terms.keys():
        doc_count += 1
    # calcula el IDF del término actual y lo almacena en el diccionario de IDFs
    doc_idfs[term] = math.log(
      float(len(doc_features.keys()))/
      float(1 + doc_count), 10)
  # devuelve el diccionario de IDFs
  return doc_idfs

def calculate_tf_idf(corpus):
  # crea un diccionario para almacenar los términos y su frecuencia en cada documento
  doc_features = {}
  # crea una lista para almacenar todos los términos únicos en todos los documentos
  vocabulary = []
  # itera por cada documento en el corpus
  for doc_id, doc in corpus.items():
    # crea un diccionario vacío para almacenar los términos y su frecuencia en el documento actual
    term_freq = {}
    # itera por cada palabra en el documento actual
    for word in doc.split():
      # si la palabra actual ya está en el diccionario de términos y frecuencias, aumenta su frecuencia en 1
      if word in term_freq.keys():
        term_freq[word] += 1
      # si la palabra actual no está en el diccionario de términos y frecuencias, agrega una nueva entrada con una frecuencia de 1
      else:
        term_freq[word] = 1
        # si la palabra actual no está en la lista de vocabulario, agréguela
        if word not in vocabulary:
          vocabulary.append(word)
    # agrega el diccionario de términos y frecuencias del documento actual al diccionario de características de documentos
    doc_features[doc_id] = term_freq
  
  # calcula los valores IDF para cada término en el vocabulario
  idfs = calculate_idfs(vocabulary, doc_features)
  
  # crea un diccionario para almacenar los vectores de características de cada documento
  doc_vectors = {}
  # itera por cada documento en el corpus
  for doc_id, doc in corpus.items():
    # crea un vector de características vacío para el documento actual
    doc_vector = []
    # itera por cada término en el vocabulario
    for term in vocabulary:
      # si el término actual está presente en el documento actual, calcula su puntaje TF-IDF y lo agrega al vector de características
      if term in doc_features[doc_id].keys():
        tf_idf = doc_features[doc_id][term] * idfs[term]
        doc_vector.append(tf_idf)
      # si el término actual no está presente en el documento actual, agrega un 0 al vector de características
      else:
        doc_vector.append(0)
    # agrega el vector de características del documento actual al diccionario de vectores de características
    doc_vectors[doc_id] = doc_vector
  
  # devuelve el diccionario de vectores de características
  return doc_vectors

corpus = {
  'doc1': 'hello world',
  'doc2': 'hello python',
  'doc3': 'python is awesome',
  'doc4': 'world is awesome'
}

doc_vectors = calculate_tf_idf(corpus)
print(doc_vectors)

: 